**1) Offline Part**

In [ ]:
##############################################  Offline Part ###################################################

import json


board_pin_ds_path = ""  #path of the board_pin dataset (.json)
pin_img_ds_path = ""    #path of the pin_image dataset (.json)
with open(board_pin_ds_path) as f:
    board_pin_ds = json.load(f)
    f.close()


with open(pin_img_ds_path) as f:
    pin_img_ds = json.load(f)
    f.close()

In [ ]:
board_IDs_list = list()
for board_pin in board_pin_ds:
   board_ID = board_pin['board_id']
   board_IDs_list.append(board_ID)

distinct_board_IDs_list = list(set(board_IDs_list))
image_names_list = list()
for pin_img in pin_img_ds:
   image_name = pin_img['im_name']
   image_names_list.append(image_name)

distinct_image_names_list = list(set(image_names_list))
board_IDs_list.clear();
image_names_list.clear();

In [ ]:
import networkx as nx

g0 = nx.Graph()
for img in pin_img_ds:
  g0.add_edge(img['im_name'],img['pin_id'])


pin_img_ds.clear();

In [ ]:

for board_obj in board_pin_ds:
  for pin in board_obj['pins'] :
      g0.add_edge(board_obj['board_id'],pin)

board_pin_ds.clear();

In [ ]:
#create image-board file
image-board-path = ""   #select a path to save the image_board edge lists (.txt)
f = open(image-board-path, "w")

g1 = nx.Graph()
for img in distinct_image_names_list :
  for neighbor in  g0.neighbors(img) :
     for x in g0.neighbors(neighbor) :
       if x!=img :
        f.write(f"{img} {x}\n")
        g1.add_edge(img,x)

f.close()
g0.clear();


In [ ]:
records = []
for board_ID in distinct_board_IDs_list:
    records.append([x for x in g1.neighbors(board_ID) if x!=board_ID])

g1.clear();
distinct_image_names_list.clear()
distinct_board_IDs_list.clear()


In [ ]:
#find Strongly pair Images in the boards  by FpGrowth#
from mlxtend.frequent_patterns import fpgrowth , association_rules
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

#Transform List of Lists to DataFrame
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)

del te
del te_ary

In [ ]:
records.clear()

df2= fpgrowth(df, min_support=0.00015, use_colnames=True)

del df

In [ ]:
min_confidence = 0.5
min_left = 1

rules = association_rules(df2 , metric ="lift", min_threshold = 1)

del df2

rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules = rules[rules['confidence'].apply(lambda x: x > min_confidence) & rules['antecedents'].apply(lambda x: len(x) == 1) & rules['consequents'].apply(lambda x: len(x) == 1)]


In [ ]:
image-image-path = ""  #select a path to save the image_image edge lists  (.txt)
f = open(image-image-path, "w")

for index,row in rules.iterrows() :
  f.write(f"{next(iter(row['antecedents']))}  {next(iter(row['consequents']))}\n")
f.close()

**2) Online Part**

In [ ]:

##############################################  Online Part ###################################################


import random
import numpy as np
import networkx as nx


image_list = []
board_list = []
gf1 = nx.Graph()

image-board-path = ""  #path of the image_board edge lists (.txt)
f = open("image-board-path", "r")

edgeList = f.readlines()
for line in edgeList:
    nodes = line.split()
    gf1.add_edge(nodes[0],nodes[1])
    image_list.append(nodes[0])
    board_list.append(nodes[1])

f.close()

image_list = list(set(image_list))
board_list = list(set(board_list))

In [ ]:

import json

cat_itr_dict = {}
board-category-path = ""  #path of the board_category dataset (.json)
with open(board-category-path) as f:
    board_cate_ds = json.load(f)
    f.close()

cat_list = []

gf_board_cat = nx.Graph()
for board_cat in board_cate_ds :
  gf_board_cat.add_edge(board_cat['board_id'] , board_cat['cate_id'])
  cat_list.append(board_cat['cate_id'])



cat_list = list(set(cat_list))

for cat in cat_list :
  cat_itr_dict[cat] = 0


cat_itr_dict = dict(sorted(cat_itr_dict.items()))


In [ ]:

image-image-path = ""  #path of the image_image edge lists (.txt)
gf2 = nx.read_edgelist(image-image-path,create_using=nx.DiGraph(), nodetype = str)



In [ ]:
def generate_random_walk(graph1,q,alpha,N):

    random_walk_list = []

    currPin = q
    steps = 0
    walk = [q]


    stop = False
    while(1):


      if(currPin==q) :
         currBoard = random.choice([n for n in graph1.neighbors(currPin) if len(list(graph1.neighbors(n)))!=1])
      else :
         currBoard = random.choice([n for n in graph1.neighbors(currPin)])


      currPin = random.choice( [n for n in graph1.neighbors(currBoard) if n!=q])

      walk.append(currPin)
      steps = steps + 1



      if(steps >= N):
        stop = True
        random_walk_list.append(walk)

        break



      if random.random()<alpha:
        currPin = q
        random_walk_list.append(walk)
        walk = [q]

    return random_walk_list;







In [ ]:

def random_walk(walk_list,q,N,np,nv):



    visit_dict = {}
    n_High_visited = 0
    for image in image_list :
      visit_dict[image] = 0

    currPin = q
    steps = 0

    stoping = False
    for walk in walk_list:
      for w in walk :

        if(w!=q) :

          steps = steps + 1
          visit_dict[w] =  visit_dict[w] + 1



          if(visit_dict[w]==nv):
            n_High_visited = n_High_visited + 1



          if(steps>= N or n_High_visited>=np):
            stoping = True
            break

      if(stoping):
        break

    return visit_dict,steps;





In [ ]:

import math

def quick_random_walk(walk_list ,q,N,np,nv,graph2,gama):



    visit_dict = {}
    n_High_visited = 0
    for image in image_list :
      visit_dict[image] = 0


    steps = 0

    stoping = False
    for walk in walk_list:
      for w in walk :
        delta = 1000
        if(w!=q) :

          steps = steps + 1

          last_visit_number_query = visit_dict[w]

          visit_dict[w] =  visit_dict[w] + 1

          if(w in graph2.nodes()):
            neighbor_count =len(list(graph2.neighbors(w)))
            for neighbor in graph2.neighbors(w):
              if(neighbor == q):
                continue

              last_visit_number_neighbor = visit_dict[neighbor]
              if(neighbor_count>1):
                delta = (1/math.log2(neighbor_count))
              if(delta > gama):
                delta = gama

              delta = gama
              visit_dict[neighbor] = round(visit_dict[neighbor] + delta,1)
              if(visit_dict[neighbor]>=nv and last_visit_number_neighbor<nv):
                n_High_visited = n_High_visited + 1



          if(visit_dict[w]>=nv and last_visit_number_query<nv):
             n_High_visited = n_High_visited + 1



          if(steps>= N or n_High_visited>=np):
            stoping = True
            break
      if(stoping):
        break

    return visit_dict,steps;



In [ ]:

def pixie_randomwalk_multiple_query(walk_list,query_set,weight_set,graph1,N,Np,Nv):
  max_degree = max([d for n, d in graph1.degree(image_list)])
  V = {}
  Vq = []
  sq = []
  sigma_sq = 0

  pixie_total_step  = 0


  for i in range(len(query_set)) :
    query_degree = 0
    query_degree =  graph1.degree(query_set[i])

    res = query_degree * (max_degree - math.log(query_degree,2))

    sq.append(res)
    sigma_sq = sigma_sq + sq[i]


  for i in range(len(query_set)) :
     Nq = math.floor(weight_set[i] * N * sq[i] / sigma_sq)

     result , total_step = random_walk(walk_list[i],query_set[i],Nq,Np,Nv)
     Vq.append(result)
     pixie_total_step = pixie_total_step + total_step


  for image in image_list:
    sum = 0
    for i in range(len(query_set)) :
      sum = sum +  math.sqrt(Vq[i][image])
    V[image] = math.pow(sum,2)


  V = dict(sorted(V.items(), key=lambda item: item[1],reverse=True))

  return V,pixie_total_step;



In [ ]:

def quick_pixie_randomwalk_multiple_query(walk_list,query_set,weight_set,graph1,N,Np,Nv,graph2,gama):
  max_degree = max([d for n, d in graph1.degree(image_list)])
  V = {}
  Vq = []
  sq = []
  query_degree = []
  sigma_sq = 0

  quick_pixie_total_step = 0

  for i in range(len(query_set)) :
    query_degree.append( graph1.degree(query_set[i]))

    res = query_degree[i] * (max_degree - math.log(query_degree[i],2))

    sq.append(res)
    sigma_sq = sigma_sq + sq[i]


  for i in range(len(query_set)) :
     Nq = math.floor(weight_set[i] * N * sq[i] / sigma_sq)

     result,total_step = quick_random_walk(walk_list[i],query_set[i],Nq,Np,Nv,graph2,gama)
     Vq.append(result)
     quick_pixie_total_step = quick_pixie_total_step + total_step


  for image in image_list:
    sum = 0
    for i in range(len(query_set)) :
      sum = sum +  math.sqrt(Vq[i][image])
    V[image] = math.pow(sum,2)

  V = dict(sorted(V.items(), key=lambda item: item[1],reverse=True))
  return V,quick_pixie_total_step




In [ ]:
# ( First Comprision)  calculate jacard index
def calculate_jaccard_index(first, second):
     jacard_index = len(set(first).intersection(second)) / len(set(first).union(second))
     return jacard_index

In [ ]:
#( second Comprision) jacard distance with query list
def category_Index(image_list,img_board_graph,board_cat_graph):
  image_cate_set = set()
  for img in image_list :
    board_list = list(gf1.neighbors(img))
    for board in board_list :
      cats = list(gf_board_cat.neighbors(board))[0]
      image_cate_set.add(cats)
  return list(image_cate_set)

def calculate_category_index(q_list,pixie_im_list,q_pixie_im_list,img_board_graph,board_cat_graph) :
  query_cat = category_Index(q_list,img_board_graph,board_cat_graph)
  pixie_keys_cat = category_Index(pixie_im_list,img_board_graph,board_cat_graph)
  quick_pixie_keys_cat = category_Index(q_pixie_im_list,img_board_graph,board_cat_graph)

  jacard_index_cat1 = calculate_jaccard_index(pixie_keys_cat,query_cat)
  jacard_index_cat2 = calculate_jaccard_index(quick_pixie_keys_cat,query_cat)
  return jacard_index_cat1,jacard_index_cat2


In [ ]:
#( Thirth Comprision)

from collections import Counter

def weighted_category_count(image_list,weight_list,img_board_graph,board_cat_graph):

  if weight_list == 0:
    weight_list = [1 for _ in range(len(image_list))]

  image_cate_list = list()
  for i in range(len(image_list)) :

    board_list = list(img_board_graph.neighbors(image_list[i]))
    for board in board_list :
      cats = [y for y in board_cat_graph.neighbors(board)]
      result = dict(Counter(cats))
      for key,value in result.items() :
         cat_itr_dict[key] =  float(cat_itr_dict[key]) + (value*weight_list[i])



  result = cat_itr_dict.copy()


  for key in cat_itr_dict.keys() :
    cat_itr_dict[key] = 0

  return result

from numpy import dot
from numpy.linalg import norm



def cosine_similarity(first_list,second_list):
  cos_sim = dot(first_list, second_list)/(norm(first_list)*norm(second_list))
  return cos_sim;




def calculate_Cosine_similarity(q_list,pixie_im_list,q_pixie_im_list ,q_weight_list,
                                img_board_graph,board_cat_graph):

  cc1 = weighted_category_count(query_list,weight_list,img_board_graph,board_cat_graph)
  cc2 = weighted_category_count(pixie_keys,0,img_board_graph,board_cat_graph)
  cc3 = weighted_category_count(quick_pixie_keys,0,img_board_graph,board_cat_graph)

  cosine_pixie_query = cosine_similarity(list(cc1.values()),list(cc2.values()))

  cosine_q_pixie_query = cosine_similarity(list(cc1.values()),list(cc3.values()))


  return cosine_pixie_query,cosine_q_pixie_query




In [ ]:
'''
with open('/content/drive/MyDrive/Pixie/Results.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      field = ["minSup", "minConf", "N", "Np" ,"Nv",
              "gama","alpha","top_k","min_weight","max_weight",
              "query_list_length","pixie_quickPixie_JI","pixie_query_JI","quick_pixie_query_JI","pixie_CS","quick_pixie_CS",
              "total_step_distance"]

      writer.writerow(field)
'''

In [ ]:
'''
import csv

def Write_Result_in_CSV(sup, conf , N , np , nv , gama , alpha , top_k , min_weight , max_weight , query_list_length , Avg_jac_Index , Avg_jacard_index_cat1 , Avg_jacard_index_cat2 , Avg_co1 , Avg_co2 , Avg_total_step_distance) :

  with open('/content/drive/MyDrive/Pixie/Results.csv', 'a', newline='') as file:
      writer = csv.writer(file)
      field = ["minSup", "minConf", "N", "Np" ,"Nv",
              "gama","alpha","top_k","min_weight","max_weight",
              "query_list_length","pixie_query_JI","quick_pixie_query_JI","pixie_CS","quick_pixie_CS",
              "total_step_distance"]

      writer.writerow([sup, conf , N , np , nv ,
                      gama , alpha , top_k , min_weight , max_weight ,
                      query_list_length , Avg_jac_Index , Avg_jacard_index_cat1 , Avg_jacard_index_cat2 , Avg_co1 , Avg_co2 ,
                      Avg_total_step_distance])
'''


In [ ]:
import itertools

sup = 0.00015
conf = 0.5
N = 10000
np = 50
nv = 100
top_k = 10
alpha = 0.2
min_weight = 1
max_weight = 2
query_list_length = 10



query_list = random.sample(image_list,query_list_length)

#print(query_list)

weight_list = set()
while(len(weight_list)<=query_list_length) :
  weight_list.add(random.uniform(min_weight,max_weight))

weight_list = list(sorted(weight_list,reverse=True))






total_step_distance_list = list()
jac_Index_list = list()
jacard_index_cat1_list =  list()
jacard_index_cat2_list =  list()
co1_list = list()
co2_list = list()

gama_list = [x/10 for x in range(0,11)]

epochs = 5

for epoch in range(epochs) :

  print(f"------------------------- epoch = {epoch+1} -------------------------------------------")

  pixie_total_step = 0
  quick_pixie_total_step = 0

  total_step_distance_for_gama = dict()
  jac_Index_for_gama = dict()
  jacard_index_cat1_for_gama =  dict()
  jacard_index_cat2_for_gama =  dict()
  co1_for_gama = dict()
  co2_for_gama = dict()

  random_walk_list_for_queryList = []
  for q in query_list :
    random_walk_list_for_queryList.append(generate_random_walk(gf1,q,alpha,N))


  V1,pixie_total_step = pixie_randomwalk_multiple_query(random_walk_list_for_queryList,query_list,weight_list,gf1,N,np,nv)
  V1 = dict(itertools.islice(V1.items(), top_k))
  #print(V1)

  for gama in gama_list :


    print("-----------------------------------------------------------------------------")
    print(f"-------------------------gama = {gama} ----------------------------------------")
    V2,quick_pixie_total_step = quick_pixie_randomwalk_multiple_query(random_walk_list_for_queryList,query_list,weight_list,gf1,N,np,nv,gf2,gama)
    V2 = dict(itertools.islice(V2.items(), top_k))

    pixie_keys = list(V1.keys())
    quick_pixie_keys = list(V2.keys())

    total_step_distance = pixie_total_step - quick_pixie_total_step
    total_step_distance_for_gama[gama] = total_step_distance


    jac_Index = calculate_jaccard_index(pixie_keys,quick_pixie_keys)
    jac_Index_for_gama[gama] = jac_Index

    jacard_index_cat1,jacard_index_cat2 = calculate_category_index(query_list,pixie_keys,quick_pixie_keys,gf1,gf_board_cat)
    jacard_index_cat1_for_gama[gama] =  jacard_index_cat1
    jacard_index_cat2_for_gama[gama] =  jacard_index_cat2

    co1,co2 = calculate_Cosine_similarity(query_list,pixie_keys,quick_pixie_keys ,weight_list,gf1,gf_board_cat)
    co1_for_gama[gama] = co1
    co2_for_gama[gama] = co2

    #print(V2)


    print(f"pixie_total_step  = {pixie_total_step}")
    print(f"quick_pixie_total_step  = {quick_pixie_total_step}")

    print(f"total_step_distance = {total_step_distance}")


    print("--------------------------Metric1----------------------------------------")


    print(f"jacard index = {jac_Index}")

    print("--------------------------Metric2----------------------------------------")

    print(f"jacard index cat1 = {jacard_index_cat1}")
    print(f"jacard index cat2 = {jacard_index_cat2}")

    print("--------------------------Metric3----------------------------------------")


    print(f"pixie_query cosine similarity = {co1}")
    print(f"quick_pixie_query cosine similarity = {co2}")

    print("\n\n******************************************************************************")


  total_step_distance_list.append(total_step_distance_for_gama)
  jac_Index_list.append(jac_Index_for_gama)
  jacard_index_cat1_list.append(jacard_index_cat1_for_gama)
  jacard_index_cat2_list.append(jacard_index_cat2_for_gama)
  co1_list.append(co1_for_gama)
  co2_list.append(co2_for_gama)





for g in gama_list :

  sum_total_step_distance = 0
  sum_jac_Index = 0
  sum_jacard_index_cat1 = 0
  sum_jacard_index_cat2 = 0
  sum_co1 = 0
  sum_co2 = 0

  for i in range(epochs) :
    sum_total_step_distance = sum_total_step_distance + total_step_distance_list[i][g]
    sum_jac_Index = sum_jac_Index + jac_Index_list[i][g]
    sum_jacard_index_cat1 = sum_jacard_index_cat1 + jacard_index_cat1_list[i][g]
    sum_jacard_index_cat2 = sum_jacard_index_cat2 + jacard_index_cat2_list[i][g]
    sum_co1 = sum_co1 + co1_list[i][g]
    sum_co2 = sum_co2 + co2_list[i][g]




  Avg_total_step_distance = sum_total_step_distance/epochs
  Avg_jac_Index = sum_jac_Index/epochs
  Avg_jacard_index_cat1 = sum_jacard_index_cat1/epochs
  Avg_jacard_index_cat2 = sum_jacard_index_cat2/epochs
  Avg_co1 = sum_co1/epochs
  Avg_co2 = sum_co2/epochs


  print(f"-------------------------------------------------------------------------------------------")
  print(f"--------------------------Averages for gama = {g} ----------------------------------------")

  print(f"Avg_total_step_distance = {Avg_total_step_distance}")

  print(f"--------------------------Average Metric1 for gama = {g} ----------------------------------------")
  print(f"Avg_ jacard index = {Avg_jac_Index}")

  print(f"--------------------------Average Metric2 for gama = {g} ----------------------------------------")

  print(f"Avg_jacard_index_cat1 = {Avg_jacard_index_cat1}")
  print(f"Avg_jacard_index_cat2 = {Avg_jacard_index_cat2}")

  print(f"--------------------------Average Metric3  for gama = {g} ----------------------------------------")


  print(f"Avg_pixie_query cosine similarity = {Avg_co1}")
  print(f"Avg_quick_pixie_query cosine similarity = {Avg_co2}")

  #Write_Result_in_CSV(sup, conf , N , np , nv , g , alpha , top_k , min_weight , max_weight , query_list_length , Avg_jac_Index , Avg_jacard_index_cat1 , Avg_jacard_index_cat2 , Avg_co1 , Avg_co2 , Avg_total_step_distance)


